In [150]:
!pip install groq

In [151]:
import json
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
from kaggle_secrets import UserSecretsClient
import itertools
from google import genai
from groq import Groq
import re

In [152]:
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [153]:
train_data = load_json("/kaggle/input/evaluate-inference-model-alqac/alqac25_train.json")         # danh sách câu hỏi
law_data = load_json("/kaggle/input/evaluate-inference-model-alqac/alqac25_law.json")       # ánh xạ article content

# Lấy nội dung của luật từ law_id và article_id 

In [154]:
law_map = {}

for law in law_data:
    for article in law.get("articles", []):
        law_map[(law["id"], article["id"])] = article["text"]

In [155]:
def get_law_content(law_infors):
    law_content = ""
    for law_info in law_infors:
        law_id = law_info['law_id']
        article_id = law_info['article_id']
        law_content += f"Luật: {law_id}" + '\n' + law_map.get((law_id, article_id), "") + '\n'*2
    return law_content

In [156]:
# print(get_law_content([
#             {
#                 "law_id": "Luật Phòng, chống ma túy",
#                 "article_id": "32"
#             }, 
#             {
#                 "law_id": "Luật Hôn nhân và gia đình",
#                 "article_id": "3"
#             }
#         ]))

# Thiết kế prompt cho từng loại câu hỏi

In [157]:
def prompt_design(question):
    # Trích thông tin law_id và article_id (chỉ lấy phần tử đầu tiên trong danh sách relevant_articles)
    law_infors = question['relevant_articles']

    # Lấy nội dung điều luật
    law_content = get_law_content(law_infors)

    # Xử lý theo từng loại câu hỏi
    if question['question_type'] == 'Đúng/Sai':
        return (
            f"Hãy đọc điều luật dưới đây và trả lời câu hỏi Đúng/Sai bên dưới.\n\n"
            f"Chỉ trả lời bằng MỘT từ duy nhất là \"Đúng\" hoặc \"Sai\".\n"
            f"KHÔNG được giải thích.\n"
            f"KHÔNG được ghi lại nội dung câu hỏi hoặc điều luật.\n"
            f"KHÔNG thêm bất kỳ nội dung nào khác ngoài một từ duy nhất: \"Đúng\" hoặc \"Sai\".\n\n"
            f"Ví dụ:\nCâu hỏi: …\nTrả lời: Đúng\n\n"
            f"Điều luật:\n{law_content}\n\n"
            f"Câu hỏi:\n{question['text']}\n\n"
            f"Trả lời:"
        )

    elif question['question_type'] == 'Trắc nghiệm':
        choices = question['choices']
        return (
            f"Dựa vào điều luật sau, hãy trả lời câu hỏi trắc nghiệm dưới đây. "
            f"Chọn duy nhất một đáp án đúng trong các lựa chọn A, B, C, D và TRẢ LỜI CHỈ BẰNG MỘT KÝ TỰ DUY NHẤT (A, B, C hoặc D), không thêm giải thích, không ghi lại nội dung đáp án.\n\n"
            f"Ví dụ: Nếu đáp án là phương án A thì chỉ trả lời: A\n\n"
            f"Điều luật:\n{law_content}\n\n"
            f"Câu hỏi:\n{question['text']}\n\n"
            f"Lựa chọn:\n"
            f"A. {choices['A']}\n"
            f"B. {choices['B']}\n"
            f"C. {choices['C']}\n"
            f"D. {choices['D']}\n\n"
            f"Chỉ trả lời A, B, C hoặc D. Không ghi thêm gì khác.\n\n"
            f"Đáp án:"
        )

    else:  # Tự luận
        return (
            f"Dựa vào điều luật sau, hãy trả lời câu hỏi dưới đây một cách chính xác và NGẮN GỌN.\n\n"
            f"YÊU CẦU:\n"
            f"- Chỉ trích xuất đúng thông tin cần thiết từ điều luật.\n"
            f"- KHÔNG được lặp lại câu hỏi.\n"
            f"- KHÔNG thêm bất kỳ lời giải thích hay mở rộng nào.\n"
            f"- KHÔNG lặp lại nội dung điều luật.\n"
            f"- Câu trả lời chỉ bao gồm cụm từ hoặc con số trọng yếu (ví dụ: \"10 ngày\" hoặc \"3 cm x 4 cm\").\n\n"
            f"Ví dụ:\n"
            f"- Câu hỏi: Thời hạn là bao lâu?\n  → Trả lời: 10 ngày\n"
            f"- Câu hỏi: Kích thước ảnh bao nhiêu?\n  → Trả lời: 3 cm x 4 cm\n\n"
            f"Điều luật:\n{law_content}\n\n"
            f"Câu hỏi:\n{question['text']}\n\n"
            f"Trả lời:"
        )


In [158]:
def prompt_design_v2(question):
    # Trích thông tin law_id và article_id (chỉ lấy phần tử đầu tiên trong danh sách relevant_articles)
    law_infors = question['relevant_articles']

    # Lấy nội dung điều luật
    law_content = get_law_content(law_infors)

    # Xử lý theo từng loại câu hỏi
    if question['question_type'] == 'Đúng/Sai':
        return (
            f"Bạn là một chuyên gia trả lời câu hỏi nhận định đúng/sai pháp luật."
            f"Dựa vào điều luật được cung cấp, hãy xác định xem nhận định trong câu hỏi dưới đây là Đúng hay Sai.\n\n"
            f"**YÊU CẦU BẮT BUỘC:**\n"
            f"Câu trả lời của bạn CHỈ ĐƯỢC PHÉP là MỘT trong hai từ sau: \"Đúng\" hoặc \"Sai\".\n\n"
            f"--- BẮT ĐẦU DỮ LIỆU ---\n\n"
            f"**Điều luật:**\n{law_content}\n\n"
            f"**Câu hỏi:**\n{question['text']}\n\n"
            f"--- KẾT THÚC DỮ LIỆU ---\n\n"
            f"**Nhận định trên là (chỉ điền Đúng hoặc Sai):**"  # Thay "Trả lời:" bằng một câu hỏi đóng hơn
        )

    elif question['question_type'] == 'Trắc nghiệm':
        choices = question['choices']
        return (
            f"Bạn là một chuyên gia trả lời câu hỏi trắc nghiệm pháp luật. Nhiệm vụ của bạn là đọc kỹ điều luật và câu hỏi, sau đó chọn một đáp án duy nhất (A, B, C, hoặc D).\n\n"
            f"**YÊU CẦU BẮT BUỘC:** Câu trả lời cuối cùng của bạn phải là MỘT KÝ TỰ DUY NHẤT.\n\n"
            f"--- Bối cảnh ---\n"
            f"Điều luật: {law_content}\n\n"
            f"--- Câu hỏi và Lựa chọn ---\n"
            f"Câu hỏi: {question['text']}\n"
            f"A. {choices['A']}\n"
            f"B. {choices['B']}\n"
            f"C. {choices['C']}\n"
            f"D. {choices['D']}\n\n"
            f"--- Đáp án ---\n"
            f"Lựa chọn chính xác nhất là (chỉ ghi A, B, C, hoặc D):" # Tín hiệu kết thúc mạnh mẽ và rõ ràng
        )

    else: # Tự luận
        return (
            f"Bạn là một trợ lý pháp lý chuyên trích xuất thông tin. Nhiệm vụ của bạn là đọc điều luật được cung cấp và trả lời câu hỏi một cách ngắn gọn, chính xác.\n\n"
            f"### YÊU CẦU NGHIÊM NGẶT:\n"
            f"1. Chỉ trích xuất phần nội dung trả lời trực tiếp cho câu hỏi trong điều luật.\n"
            f"2. KHÔNG được sao chép toàn bộ đoạn luật, chỉ lấy phần có ý nghĩa trả lời.\n"
            f"3. KHÔNG được viết lại câu hỏi, không được thêm giải thích hoặc bình luận.\n"
            f"4. Câu trả lời phải ngắn gọn, đầy đủ ý và đúng theo điều luật.\n\n"
            f"<DỮ LIỆU>\n"
            f"Điều luật:\n{law_content}\n\n"
            f"Câu hỏi:\n{question['text']}\n\n"
            f"<Câu trả lời (chỉ một câu ngắn, không dài dòng, không lặp lại thông tin)> \n"
        )


# Sử dụng LLM để trả lời câu hỏi

In [159]:
def predict_answer(model, tokenizer, question):
    input_text = prompt_design_v2(question)
    
    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Thiết lập cấu hình tùy theo loại câu hỏi
    qtype = question['question_type']

    if qtype == "Đúng/Sai":
        do_sample = False
        max_new_tokens = 3  # Đủ để sinh ra "Đúng" hoặc "Sai"

    elif qtype == "Trắc nghiệm":
        do_sample = False
        max_new_tokens = 3  # Đủ để sinh ra "A", "B", "C", "D"

    else:
        do_sample = False # Có thể để True nếu cần sinh ra câu trả lời đa dạng
        max_new_tokens = 128  # Đủ dài để trả lời ngắn gọn nhưng đầy đủ
        
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=0.1 if do_sample else 1.0,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode only the generated part (exclude input)
    generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    return response


# Tải mô hình và tokenizer

In [160]:
model_name = "google/gemma-7b"

In [ ]:
# # Load model and tokenizer
# from huggingface_hub import login


# print(f"Loading {model_name} model...")

# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     trust_remote_code=True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
#     device_map="auto" if torch.cuda.is_available() else None,
#     trust_remote_code=True,
#     use_cache=True
# )

# # Set pad token if not exists
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# print("Model loaded successfully!")
# print(f"Model device: {model.device}")
# print(f"Model dtype: {model.dtype}")

# Xây dựng pipeline đánh giá model từ train data

In [162]:
def run_pipeline(model, tokenizer):
    results = []
    for question in tqdm(train_data, desc="Processing and Evaluating questions"):
        # if question['question_type'] == "Trắc nghiệm": # Test
            try:
                # 1. Dự đoán từ mô hình của bạn
                pred_answer = predict_answer(model, tokenizer, question)
                ground_truth_answer = question.get("answer", None)
    
                # # 2. Đánh giá Đúng/Sai bằng API
                # evaluation_result = get_binary_evaluation(pred_answer, ground_truth_answer)
                
                # 3. Ghi kết quả
                results.append({
                    "question_id": question["question_id"],
                    "question": question["text"],
                    "question_type": question["question_type"],
                    "predicted_answer": pred_answer,
                    "ground_truth_answer": ground_truth_answer,
                    # "evaluation": evaluation_result
                })
                
            except Exception as e:
                print(f"Lỗi trong pipeline với câu hỏi {question.get('question_id', 'unknown')}: {str(e)}")
                results.append({
                    "question_id": question.get("question_id", "unknown"),
                    "question": question["text"],
                    "question_type": question["question_type"],
                    "predicted_answer": "ERROR",
                    "ground_truth_answer": question.get("answer", None),
                    # "evaluation": "Lỗi_pipeline"
                })
            # break # Test
    
    return results

# RUN

In [163]:
# # Run the pipeline
# print("Starting evaluation pipeline...")
# final_results = run_pipeline(model, tokenizer)

# # Save results
# output_file = f"{model_name.split('/')[-1]}_results.json"
# with open(output_file, 'w', encoding='utf-8') as f:
#     json.dump(final_results, f, indent=2, ensure_ascii=False)

# print(f"Evaluation completed! Results saved to {output_file}")
# print(f"Total questions processed: {len(final_results)}")

In [164]:
# # Print some sample results
# print("\nSample results:")
# for i, result in enumerate(final_results[:3]):
#     print(f"\nQuestion {i+1}:")
#     print(f"Type: {result['question_type']}")
#     print(f"Question: {result['question'][:100]}...")
#     print(f"Predicted: {result['predicted_answer']}")
#     print(f"Ground Truth: {result['ground_truth_answer']}")

# Gọi API Gemini và Groq để đánh giá kết quả

In [165]:
# Lấy API key
user_secrets = UserSecretsClient()

In [166]:
gemini_keys = [
    user_secrets.get_secret(f"GEMINI_API_KEY_{i}") for i in range(1, 4)
]

gemini_cycle = itertools.cycle(gemini_keys)

In [167]:
def get_next_gemini_key():
    return next(gemini_cycle)

In [168]:
def call_gemini(prompt, api_key):
    client_gemini = genai.Client(api_key = api_key) 
    response = client_gemini.models.generate_content(
            model="gemini-2.5-flash", 
            contents=prompt
    )
    return response.text

In [169]:
# print(call_gemini("Trường Đại học Công Nghệ Thông tin?", get_next_gemini_key()).strip())

In [170]:
groq_keys = [
    user_secrets.get_secret(f"GROQ_API_KEY_{i}") for i in range(1, 4)
]

groq_cycle = itertools.cycle(groq_keys)

In [171]:
def get_next_groq_key():
    return next(groq_cycle)

In [172]:
def call_groq(prompt, api_key):
    client_groq = Groq(
        api_key=api_key,
    )
    
    chat_completion = client_groq.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="qwen/qwen3-32b",
    )
    
    return chat_completion.choices[0].message.content

In [173]:
# print(call_groq("Trường Đại học Công Nghệ Thông tin?", get_next_groq_key()).strip())

# Prompt để đánh giá kết quả

In [174]:
# --- ĐỊNH NGHĨA PROMPT ---
EVALUATION_PROMPT_TEMPLATE = """**Bối cảnh:** Bạn là một giám khảo cực kỳ nghiêm khắc. Nhiệm vụ của bạn là so sánh "Câu trả lời dự đoán" và "Câu trả lời đúng (Ground Truth)".

**Quy tắc:**
- Nếu "Câu trả lời dự đoán" có cùng ý nghĩa hoặc tương đương về mặt ngữ nghĩa với "Câu trả lời đúng", hãy coi nó là **Đúng**.
- Nếu "Câu trả lời dự đoán" sai về mặt thông tin, hoặc thiếu thông tin cốt lõi, hãy coi nó là **Sai**.

**Yêu cầu về định dạng đầu ra:**
TUYỆT ĐỐI chỉ trả lời MỘT TỪ DUY NHẤT: **Đúng** hoặc **Sai**.
KHÔNG giải thích. KHÔNG thêm bất kỳ văn bản hay ký tự nào khác.

---
**VÍ DỤ 1:**
**Câu trả lời dự đoán:** Hồ sơ đề nghị cấp lại thẻ hướng dẫn viên du lịch bao gồm ảnh chân dung màu cỡ 3 cm x 4 cm.
**Câu trả lời đúng (Ground Truth):** 3 cm x 4 cm.
**Đánh giá:**
Đúng

---
**VÍ DỤ 2:**
**Câu trả lời dự đoán:** Lừa dối, đe dọa, cưỡng ép, mua chuộc, sử dụng vũ lực nhằm ngăn cản người phiên dịch thực hiện nhiệm vụ hoặc buộc người phiên dịch dịch không trung thực, không khách quan, không đúng nghĩa..
**Câu trả lời đúng (Ground Truth):** Dịch sai sự thật.
**Đánh giá:**
Sai

---
**YÊU CẦU ĐÁNH GIÁ:**
**Câu trả lời dự đoán:**
{predicted_answer}

**Câu trả lời đúng (Ground Truth):**
{ground_truth_answer}

**Đánh giá:**
"""

# So sánh Predict Answer và Ground Truth Answer

## Dùng Regex cho câu hỏi Đúng/Sai

In [175]:
def check_true_false(predicted, ground_truth):
    # Tìm tất cả các từ "Đúng" hoặc "Sai"
    matches = re.findall(r'\b(Đúng|Sai)\b', predicted, re.IGNORECASE)
    matches = [m.capitalize() for m in matches]
    matches_set = set(matches)

    # Không có câu trả lời hợp lệ
    if not matches_set:
        return 0

    # Có cả Đúng và Sai cùng lúc ⇒ sai ngay
    if 'Đúng' in matches_set and 'Sai' in matches_set:
        return 0

    # Nếu chỉ có một trong hai và khớp ground_truth ⇒ đúng
    if ground_truth in matches_set:
        return 1
    else:
        return 0

In [176]:
print(check_true_false("Sai\n\n---", "Sai"))
print(check_true_false("Đúng", "Đúng"))
print(check_true_false("Sai\n\n---", "Đúng"))
print(check_true_false("Đúng", "Sai"))


1
1
0
0


## Dùng Regex cho câu hỏi Trắc nghiệm

In [177]:
def check_multiple_choice(predicted, ground_truth):
    # Kiểm tra có ít nhất một ký tự A, B, C, D hay không
    if not re.search(r'\b[A-D]\b', predicted, re.IGNORECASE):
        return 0

    # Duyệt từ trái sang phải để tìm ký tự đầu tiên A-D
    for ch in predicted.upper():
        if ch in {'A', 'B', 'C', 'D'}:
            return 1 if ch == ground_truth.upper() else 0
    return 0

In [178]:
print(check_multiple_choice("A\n\n---", "A"))
print(check_multiple_choice("C\nC", "C"))
print(check_multiple_choice("A\n\n---", "A"))
print(check_multiple_choice("A\n\nA", "B"))

1
1
1
0


## Gọi API cho câu hỏi tự luận

In [179]:
def check_free_text(pred_answer, ground_truth_answer, i):
    """
    Sử dụng Gemini/Groq để trả về 'Đúng' hoặc 'Sai'.
    """
    if not pred_answer or not ground_truth_answer:
        return 0  # Lỗi đầu vào coi là sai

    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        predicted_answer=pred_answer,
        ground_truth_answer=ground_truth_answer
    )
    
    try:
        if i % 2 == 0:
            response = call_gemini(prompt, get_next_gemini_key())
        else:
            response = call_groq(prompt, get_next_groq_key())
        
        result_text = response.strip()

        # Chuẩn hóa kết quả
        if result_text == "Đúng":
            return 1
        elif result_text == "Sai":
            return 0
        else:
            # Trường hợp Gemini/Groq trả về không sạch ⇒ fallback
            print(f"Warning: Gemini/Groq response unclear")
            if "Đúng" in result_text:
                return 1
            elif "Sai" in result_text:
                return 0
            else:
                return 0  # Không xác định ⇒ mặc định sai

    except Exception as e:
        print(f"Lỗi khi gọi API: {str(e)}")
        time.sleep(1)
        return 0  # Lỗi hệ thống ⇒ mặc định sai

In [180]:
print(check_free_text(
    "Thông tin thuộc bí mật nhà nước được tiếp cận khi được giải mật theo quy định của Luật này.\nDựa trên điều luật, thông tin thuộc bí mật nhà nước được tiếp cận khi được giải mật theo quy định của Luật Tiếp cận thông tin. \n\n(Câu trả lời đã tuân thủ yêu cầu, ngắn gọn và chính xác)",
    "được giải mật",
    0
))

1


# Đánh giá bằng API

In [181]:
def evaluate_accuracy(file_path, output_path):
    with open(file_path, "r", encoding='utf-8') as f:
        data = json.load(f)

    correct_list = []
    total = len(data)

    print(f"Tổng số câu hỏi: {total}")

    for i, item in enumerate(data):
        qtype = item.get('question_type', '').strip()
        pred = item.get('predicted_answer', '').strip()
        gt = item.get('ground_truth_answer', '').strip()

        if qtype == 'Đúng/Sai':
            is_true = check_true_false(pred, gt)
        elif qtype == 'Trắc nghiệm':
            is_true = check_multiple_choice(pred, gt)
        elif qtype == 'Tự luận':
            is_true = check_free_text(pred, gt, i)
            
        else:
            is_true = 0  # Unknown question type

        item['is_True'] = is_true
        correct_list.append(is_true)

    accuracy = sum(correct_list) / total if total > 0 else 0
    
    print(f"Accuracy tổng thể: {accuracy:.2%}")
    
    with open(output_path, "w", encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"Kết quả đã lưu vào: {output_path}")


In [202]:
file_path = "/kaggle/input/model-results/inference_result/Llama-3.1-8B-Instruct_results.json" 
file_name = file_path.split('/')[-1]
output_path = f"/kaggle/working/{file_name[:-5]}_CHECK.json"
print(output_path)

/kaggle/working/Llama-3.1-8B-Instruct_results_CHECK.json


In [203]:
evaluate_accuracy(file_path, output_path)

Tổng số câu hỏi: 729
Accuracy tổng thể: 73.80%
Kết quả đã lưu vào: /kaggle/working/Llama-3.1-8B-Instruct_results_CHECK.json
